In [161]:
import numpy as np 
import pandas as pd 
from pandas.io.json import json_normalize 
import json 
import requests 
from bs4 import BeautifulSoup # scraping library
print('Libraries imported.')

Libraries imported.


In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

text_result = requests.get(url).text #get the entire html of the article as a str
html_parsed_result = BeautifulSoup(text_result, 'html.parser') 
neighborhood_info_table = html_parsed_result.find('table', class_ = 'wikitable')
neighborhood_rows = neightborhood_info_table.find_all('tr')

In [ ]:
# extract the info ('Postcode', 'Borough', 'Neighbourhood') from the table
neighborhood_info = []
for row in neighborhood_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    neighborhood_info.append(info)

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [153]:
#create a dataframe
neighborhood_df = pd.DataFrame(neighborhood_info[1:], columns=neightborhood_info[0])

In [154]:
#Search the 'Not assigned' values
not_assigned_boroughs = neighborhood_df.index[neighborhood_df['Borough'] == 'Not assigned']
not_assigned_neighborhoods = neighborhood_df.index[neighborhood_df['Neighbourhood'] == 'Not assigned']
not_assigned_neighborhoods_and_borough = not_assigned_boroughs & not_assigned_neighborhoods

print('The DataFrame shape is {}'.format(neighborhood_df.shape),'\n')
print('------------------------------------------------------------\n')
print('There are:')
print('  {} Postal codes'.format(neighborhood_df['Postcode'].unique().shape[0]))
print('  {} Boroughs'.format(neighborhood_df['Borough'].unique().shape[0] - 1)) # substract one because "not assigned" doesn't count
print('  {} Neighborhoods'.format(neighborhood_df['Neighbourhood'].unique().shape[0] - 1)) # substract one because "not assigned" doesn't count
print('  {} rows with Not assigned Borough'.format(not_assigned_boroughs.shape[0]))
print('  {} rows with Not assigned Neighbourhood'.format(not_assigned_neighborhoods.shape[0]))
print('  {} rows with Not assigned Neighbourhood and Borough'.format(not_assigned_neighborhoods_and_borough.shape[0]),'\n')
print('------------------------------------------------------------\n')


The DataFrame shape is (287, 3) 

------------------------------------------------------------

There are:
  180 Postal codes
  11 Boroughs
  207 Neighborhoods
  77 rows with Not assigned Borough
  78 rows with Not assigned Neighbourhood
  77 rows with Not assigned Neighbourhood and Borough 

------------------------------------------------------------



Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [155]:
neighborhood_df.drop(neighborhood_df.index[not_assigned_boroughs], inplace=True)
neighborhood_df.reset_index(drop=True, inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [156]:
not_assigned_neighborhoods = neighborhood_df.index[neighborhood_df['Neighbourhood'] == 'Not assigned'] # reset the index

for idx in not_assigned_neighborhoods:
    neighborhood_df['Neighbourhood'][idx] = neighborhood_df['Borough'][idx]
    
neighborhood_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [157]:
print('After cleaning the DataFrame, its new shape is {}'.format(neighborhood_df.shape),'\n')
print('There are:')
print('  {} Postal codes with repetition'.format(neighborhood_df.Postcode.shape[0]))
print('  {} Postal codes without repetition'.format(neighborhood_df['Postcode'].unique().shape[0]))
print('  {}  Boroughs'.format(neighborhood_df['Borough'].unique().shape[0]))
print('  {} Neighborhoods'.format(neighborhood_df['Neighbourhood'].unique().shape[0]))

After cleaning the DataFrame, its new shape is (210, 3) 

There are:
  210 Postal codes with repetition
  103 Postal codes without repetition
  11  Boroughs
  207 Neighborhoods


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [159]:
group = neighborhood_df.groupby('Postcode')
grouped_neighborhoods = group['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
grouped_df.columns = ['Postcode', 'Borough', 'Neighbourhood']

grouped_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [160]:
print('The DataFrame shape is', grouped_df.shape)

The DataFrame shape is (103, 3)
